In [2]:
!pip install -U git+https://github.com/cseveriano/fts2image@googlenet-testing

  Cloning https://github.com/cseveriano/fts2image (to revision googlenet-testing) to /tmp/pip-req-build-hs_odvb9
Branch 'googlenet-testing' set up to track remote branch 'googlenet-testing' from 'origin'.
Switched to a new branch 'googlenet-testing'
  Running setup.py bdist_wheel for fts2image ... - done
  Stored in directory: /tmp/pip-ephem-wheel-cache-bn071tu8/wheels/d8/4f/00/2939bfdf8fdead327ea7c1aa05ac9d0212e1639a00c1502dae
Successfully built fts2image


In [3]:
!pip install -U git+https://github.com/petroniocandido/pyFTS

  Cloning https://github.com/petroniocandido/pyFTS to /tmp/pip-req-build-dukczq54
  Running setup.py bdist_wheel for pyFTS ... - \ done
  Stored in directory: /tmp/pip-ephem-wheel-cache-ui3vcaku/wheels/84/d7/1e/a333c7128f25b347640740859808db094c4478e98663cd2297
Successfully built pyFTS


In [6]:
!pip install -U hyperopt

    100% |████████████████████████████████| 122kB 5.6MB/s 
    100% |████████████████████████████████| 409kB 7.0MB/s 


In [4]:
import pandas as pd
import os
from pyFTS.partitioners import Grid, Entropy, Util as pUtil
import matplotlib.pyplot as plt
import math
from sklearn.metrics import mean_squared_error
import pickle
from fts2image import FuzzyImageCNN


Using TensorFlow backend.


In [0]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt import space_eval

## Auxiliary Functions

In [0]:
def normalize(df):
    mindf = df.min()
    maxdf = df.max()
    return (df-mindf)/(maxdf-mindf)

def denormalize(norm, _min, _max):
    return [(n * (_max-_min)) + _min for n in norm]

def calculate_rmse(test, forecast, order, step):
    rmse = math.sqrt(mean_squared_error(test.iloc[(order):], forecast[:-step]))
    print("RMSE : "+str(rmse))
    return rmse

def split_data(df, interval):
    sample_df = df.loc[interval]

    week = (sample_df.index.day - 1) // 7 + 1

    # PARA OS TESTES:
    # 2 SEMANAS PARA TREINAMENTO
    train_df = sample_df.loc[week <= 2]

    # 1 SEMANA PARA VALIDACAO
    validation_df = sample_df.loc[week == 3]

    # 1 SEMANA PARA TESTES
    test_df = sample_df.loc[week > 3]

    return (train_df, validation_df, test_df)

In [0]:
def calculate_rmse(test, forecast, order, step):
    rmse = math.sqrt(mean_squared_error(test.iloc[(order):], forecast[:-step]))
    print("RMSE : "+str(rmse))
    return rmse

## Load Dataset

In [0]:
from urllib import request

#Set target and input variables 
target_station = 'DHHL_3'

#All neighbor stations with residual correlation greater than .90
neighbor_stations_90 = ['DHHL_3',  'DHHL_4','DHHL_5','DHHL_10','DHHL_11','DHHL_9','DHHL_2', 'DHHL_6','DHHL_7','DHHL_8']

dataset_path = "https://github.com/cseveriano/spatio-temporal-forecasting/raw/master/notebooks/"

request.urlretrieve(dataset_path+"df_oahu.pkl", "df_oahu")

df = pd.read_pickle("df_oahu")

# Save Min-Max for Denorm
min_raw = df[target_station].min()

max_raw = df[target_station].max()

# Perform Normalization
norm_df = normalize(df)

# Split data
interval = ((df.index >= '2010-06') & (df.index < '2010-07'))
#interval = ((df.index >= '2010-11') & (df.index <= '2010-12'))

(train_df, validation_df, test_df) = split_data(df, interval)
(norm_train_df, norm_validation_df, norm_test_df) = split_data(norm_df, interval)

## CNN Parameter Tuning

In [0]:
###### CNN FUNCTIONS ###########
def fuzzy_cnn_forecast(train_df, test_df, params):
    fuzzy_sets = Grid.GridPartitioner(data=train_df[target_station].values, npart=params['npartitions']).sets
    model = FuzzyImageCNN.FuzzyImageCNN(fuzzy_sets, nlags=params['order'], steps=1,
            conv_layers = params['conv_layers'], dense_layers = params['dense_layers'],
            filters = params['filters'], kernel_size = params['kernel_size'],
            pooling_size = params['pooling_size'], dense_layer_neurons = params['dense_layer_neurons'], dropout=params['dropout'])

    model.fit(train_df, epochs=params['epochs'])

    forecast = model.predict(test_df)

    return forecast


In [0]:
def cnn_objective(params):
    print(params)
    try:
        forecast = fuzzy_cnn_forecast(norm_train_df[neighbor_stations_90], norm_test_df[neighbor_stations_90], params)
        forecast = denormalize(forecast, min_raw, max_raw)
        forecast.append(0) ## para manter o mesmo tamanho dos demais
        rmse = calculate_rmse(test_df[target_station], forecast, params['order'], 1)
    except Exception:
        traceback.print_exc()
        rmse = 1000

    return {'loss': rmse, 'status': STATUS_OK}


In [0]:
###### OPTIMIZATION ROUTINES ###########
space = {'npartitions': hp.choice('npartitions', [50, 100, 150, 200]),
        'order': hp.choice('order', [96,144]),
        'epochs': hp.choice('epochs', [30, 50, 100]),
        'conv_layers' : hp.choice('conv_layers', list(np.arange(2,6))),
        'dense_layers': hp.choice('dense_layers', list(np.arange(2, 6))),
        'filters': hp.choice('filters',  [2, 4, 8, 16, 32, 64]),
        'kernel_size': hp.choice('kernel_size', list(np.arange(2,5))),
        'pooling_size': hp.choice('pooling_size', list(np.arange(2, 5))),
        'dense_layer_neurons': hp.choice('dense_layer_neurons', [32, 64, 128, 256, 512, 768, 1024, 1280]),
        'dropout': hp.choice('dropout', list(np.arange(0.2, 0.5, 0.1)))}

In [18]:
trials = Trials()
best = fmin(cnn_objective, space, algo=tpe.suggest, max_evals =500, trials=trials)

{'conv_layers': 3, 'dense_layer_neurons': 768, 'dense_layers': 4, 'dropout': 0.4000000000000001, 'epochs': 50, 'filters': 8, 'kernel_size': 3, 'npartitions': 50, 'order': 144, 'pooling_size': 4}
Epoch 1/1
710/710 [==============================] - 3s 4ms/step - loss: 0.0625
RMSE : 244.1839754279612
{'conv_layers': 5, 'dense_layer_neurons': 256, 'dense_layers': 3, 'dropout': 0.4000000000000001, 'epochs': 100, 'filters': 8, 'kernel_size': 2, 'npartitions': 200, 'order': 144, 'pooling_size': 4}
Epoch 1/1
710/710 [==============================] - 5s 8ms/step - loss: 0.1010
RMSE : 311.5336994448155
{'conv_layers': 5, 'dense_layer_neurons': 768, 'dense_layers': 2, 'dropout': 0.2, 'epochs': 50, 'filters': 4, 'kernel_size': 4, 'npartitions': 50, 'order': 144, 'pooling_size': 3}
Epoch 1/1
710/710 [==============================] - 3s 4ms/step - loss: 0.0809
RMSE : 272.3190153264894
{'conv_layers': 2, 'dense_layer_neurons': 256, 'dense_layers': 4, 'dropout': 0.4000000000000001, 'epochs': 100, '

NameError: ignored

In [0]:
print('best: ')
print(space_eval(space, best))

In [0]:
pickle.dump(best, open("best_result.pkl", "wb"))
pickle.dump(trials, open("tuning_results.pkl", "wb"))